In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments, MBartForConditionalGeneration
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import torch, os

/home/huynv/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
project_folder = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
data_folder = os.path.join(project_folder, 'data')
checkpoint_folder = os.path.join(project_folder, 'checkpoint')
print(project_folder, data_folder, checkpoint_folder)

c:\Users\Huy\GitHub\NLP c:\Users\Huy\GitHub\NLP\data c:\Users\Huy\GitHub\NLP\checkpoint


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable") 
model = AutoModelForSeq2SeqLM.from_pretrained("vinai/bartpho-syllable").to(device)

In [5]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=256, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [6]:
df_train = pd.read_csv(os.path.join(data_folder, 'vietnews_train.csv'))[:25000]

In [7]:
df_train = df_train[["original", "summary"]]
df_train.rename(columns = {'original':'inputs'}, inplace = True)
df_train.rename(columns = {'summary':'labels'}, inplace = True)

In [8]:
df_train

,inputs,labels
0,Bộ_trưởng Sitharaman phát_biểu tại buổi gặp_gỡ...,Bộ_trưởng Quốc_phòng Ấn_Độ bày_tỏ mong_muốn Vi...
1,Tổng_thống Pháp Emmanuel_Macron phấn_khích ăn_...,"Sau màn ăn_mừng phấn_khích trên khán_đài , Emm..."
2,Lực_lượng tuần_duyên Nhật_Bản đang nỗ_lực tìm_...,Một hòn đảo của Nhật nằm sát khu_vực tranh_chấ...
3,"Cảnh_sát thành_phố Barcelona , Tây_Ban_Nha đã ...",Tình_báo Mỹ báo với cảnh_sát Tây_Ban_Nha về ng...
4,"Trước_đây , khán_giả thấy rằng nghệ_sĩ Xuân_Hư...",Nghệ_sĩ Xuân_Hương và MC Thanh_Bạch đã từng đư...
...,...,...
24995,"Ngày 7-6 , Đội quản_lý thị_trường số 17 Chi_cụ...",Lực_lượng quản_lý thị_trường đã tạm giữ hàng c...
24996,"Ngày 17/6 , báo_cáo với UBND tỉnh Bình_Thuận v...",Công_ty khai_thác bơm nước về hồ nhưng không k...
24997,"Kay_Longstaff , người phụ_nữ Anh sống_sót sau ...",Kay_Longstaff bị ngã khỏi tàu du_lịch khi đang...
24998,Tổng_thống Mỹ Donald_Trump trong một cuộc diễn...,Tổng_thống Mỹ tuyên_bố nếu có cuộc đấu quyền_a...


In [9]:
dataset = Dataset.from_pandas(df_train)
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

Map (num_proc=8):   0%|          | 0/25000 [00:00<?, ? examples/s]/home/huynv/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/huynv/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map (num_proc=8):   4%|▍         | 1000/25000 [00:04<01:42, 234.00 examples/s]/home/huynv/miniconda3/lib/pytho

In [10]:

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


training_args = Seq2SeqTrainingArguments(checkpoint_folder,
                                      do_train=True,
                                      do_eval=False,
                                      num_train_epochs=2,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=1,
                                      per_device_eval_batch_size=1,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                    #   remove_unused_columns=False,
                                      #eval_steps=1,
                                      #evaluation_strategy="steps",
                                      # evaluation_strategy="no",
                                      fp16=True,
                                      )



In [11]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()